In [ ]:
import seaborn as sns
import pandas as pd

# Iris

In [ ]:
iris = sns.load_dataset('iris')

## a
붓꽃 종(species)별로 꽃잎길이(sepal_length), 꽃잎폭(sepal_width),
꽃받침길이(petal_length), 꽃받침폭(petal_width)의 평균, 표준편차 등
기초통계량(describe())을 구하시오.

## b
3분위수(Q3)와 1분위수(Q1)의 차이보다 1.5배가 크거나 작은 데이터는 이상치이다.
즉,
* Q1 - 1.5 * (Q3 - Q1) 보다 작은 데이터
* Q3 + 1.5 * (Q3 - Q1) 보다 큰 데이터

이 이상치를 제거하고 위의 4가지 항목에 대해서 평균, 표준편차를 구하시오.

# Titanic


In [ ]:
titanic = sns.load_dataset('titanic')

## a
타이타닉호의 승객에 대해 나이와 성별에 의한 카테고리 열인 category1 열을 만드시오. 
category1 카테고리는 다음과 같이 정의됨

1) 20살이 넘으면 성별을 그대로 사용한다.
2) 20살 미만이면 성별에 관계없이 “child”라고 한다.


In [ ]:
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [ ]:
titanic['category1'] =  titanic.apply(lambda x : "child" if x.age < 20 else "male" if x.sex=='male' else 'female', axis=1)
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,male
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,female
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,female
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,female
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True,male
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True,child
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False,female
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True,male


## b
 타이타닉호의 승객 중 나이를 명시하지 않은 고객은 나이를 명시한 고객의 평균 나이 값
이 되도록 titanic 데이터프레임을 고치시오.


In [ ]:
mean_age = titanic.age.mean(axis=0)
titanic.age.fillna(mean_age, inplace=True)

## c
성별, 선실(class)별, 출발지(embark_town)별 생존율을 구하시오.

In [ ]:
sex_group = titanic.groupby('sex')['survived']
sex_group.mean()

sex
female    0.742038
male      0.188908
Name: survived, dtype: float64

In [ ]:
class_group = titanic.groupby('class')['survived']
class_group.mean()

class
First     0.629630
Second    0.472826
Third     0.242363
Name: survived, dtype: float64

In [ ]:
embark_town_group = titanic.groupby('embark_town')['survived']
embark_town_group.mean()

embark_town
Cherbourg      0.553571
Queenstown     0.389610
Southampton    0.336957
Name: survived, dtype: float64

## d
타이타닉호 승객을 ‘미성년자’, ‘청년’, ‘중년’, ‘장년’, ‘노년’ 나이 그룹으로 나누고, 각 그룹별 생존율을 구하시오.
* bins = [1, 20, 30, 50, 70, 100]
* labels = ["미성년자", "청년", "중년", "장년", "노년"]


In [ ]:
bins = [0, 20, 30, 50, 70, 100]
labels = ["미성년자", "청년", "중년", "장년", "노년"]
titanic["연령대"] = pd.cut(titanic.age, bins, labels=labels) 

titanic_age = pd.DataFrame(titanic.연령대.value_counts(), columns=["연령대"]) 
titanic_age["생존율"] = titanic.groupby('연령대')['survived'].mean()

In [ ]:
titanic_age

,연령대,생존율
청년,407,0.334152
중년,241,0.423237
미성년자,179,0.458101
장년,59,0.355932
노년,5,0.200000


## e
qcut 명령으로 세 개의 나이 그룹을 만들고, 나이 그룹별 남녀 성비와 생존율을 구하시오.

In [ ]:
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1,연령대
0,0,3,male,22.000000,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,male,청년
1,1,1,female,38.000000,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,female,중년
2,1,3,female,26.000000,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,female,청년
3,1,1,female,35.000000,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,female,중년
4,0,3,male,35.000000,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,male,중년
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.000000,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True,male,청년
887,1,1,female,19.000000,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True,child,미성년자
888,0,3,female,29.699118,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False,female,청년
889,1,1,male,26.000000,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True,male,청년


In [ ]:
titanic['age-group'] = pd.qcut(titanic.age, 3, labels=['젊은이', '중간', '늙은이'])
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1,연령대,age-group
0,0,3,male,22.000000,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,male,청년,젊은이
1,1,1,female,38.000000,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,female,중년,늙은이
2,1,3,female,26.000000,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,female,청년,중간
3,1,1,female,35.000000,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,female,중년,늙은이
4,0,3,male,35.000000,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,male,중년,늙은이
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.000000,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True,male,청년,중간
887,1,1,female,19.000000,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True,child,미성년자,젊은이
888,0,3,female,29.699118,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False,female,청년,중간
889,1,1,male,26.000000,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True,male,청년,중간


In [ ]:
age_group_survived = titanic.groupby('age-group')['survived']
age_group_survived.mean()

age-group
젊은이    0.411960
중간     0.335526
늙은이    0.405594
Name: survived, dtype: float64

In [ ]:
titanic['sex_category'] = titanic.apply(lambda x : 1 if x.sex=='male' else 0, axis=1)
titanic['sex_category_f'] = titanic.apply(lambda x : 1 if x.sex=='female' else 0, axis=1)

titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1,연령대,age-group,sex_category,sex_category_f
0,0,3,male,22.000000,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,male,청년,젊은이,1,0
1,1,1,female,38.000000,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,female,중년,늙은이,0,1
2,1,3,female,26.000000,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,female,청년,중간,0,1
3,1,1,female,35.000000,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,female,중년,늙은이,0,1
4,0,3,male,35.000000,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,male,중년,늙은이,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.000000,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True,male,청년,중간,1,0
887,1,1,female,19.000000,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True,child,미성년자,젊은이,0,1
888,0,3,female,29.699118,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False,female,청년,중간,0,1
889,1,1,male,26.000000,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True,male,청년,중간,1,0


In [ ]:
# 나이 그룹별 남자비율
age_group_survived = titanic.groupby('age-group')['sex_category']
age_group_survived.mean()

age-group
젊은이    0.594684
중간     0.680921
늙은이    0.667832
Name: sex_category, dtype: float64

In [ ]:
# 나이 그룹별 여자비율
age_group_survived_f = titanic.groupby('age-group')['sex_category_f']
age_group_survived_f.mean()

age-group
젊은이    0.405316
중간     0.319079
늙은이    0.332168
Name: sex_category_f, dtype: float64

# mpg

In [1]:
mpg=sns.load_dataset('mpg')

NameError: ignored

In [ ]:
mpg

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino
...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790,15.6,82,usa,ford mustang gl
394,44.0,4,97.0,52.0,2130,24.6,82,europe,vw pickup
395,32.0,4,135.0,84.0,2295,11.6,82,usa,dodge rampage
396,28.0,4,120.0,79.0,2625,18.6,82,usa,ford ranger


#3 - A


In [ ]:
df = mpg[['displacement','horsepower']]

In [ ]:
df['hp_per_cc'] = df['displacement']/df['horsepower'].round(3)

<ipython-input-27-f46c80428f2d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['hp_per_cc'] = df['displacement']/df['horsepower'].round(3)


In [ ]:
df1 = mpg['name']
len(df1[0].split())
df1[0].split()

['chevrolet', 'chevelle', 'malibu']

#3-B

In [ ]:
def listToString(str_list):
    result = ""
    for s in str_list:
        result += s + " "
    return result.strip()

In [ ]:
df_list=[]
for i in range(len(df1)):
  if len(df1[i].split())==1:
    a = df1[i].split()
    df_list.append(a[0])
  if len(df1[i].split())==6:
    a = df1[i].split()
    df_list.append(a[0])
  if len(df1[i].split())==5:
    a = df1[i].split()
    df_list.append(a[0])
  if len(df1[i].split())==4:
    a = df1[i].split()
    df_list.append(a[0])
  if len(df1[i].split())== 3:
    a = df1[i].split()
    df_list.append(a[0])
  if len(df1[i].split())==2:
    a = df1[i].split()
    df_list.append(a[0])

  

    


In [ ]:
df_list1=[]
for i in range(len(df1)):
  if len(df1[i].split())==1:
    a = df1[i].split()
    b = listToString(a[1:])
    df_list1.append(b)
  if len(df1[i].split())==6:
    a = df1[i].split()
    b = listToString(a[1:])
    df_list1.append(b)
  if len(df1[i].split())==5:
    a = df1[i].split()
    b = listToString(a[1:])
    df_list1.append(b)
  if len(df1[i].split())==4:
    a = df1[i].split()
    b = listToString(a[1:])
    df_list1.append(b)
  if len(df1[i].split())== 3:
    a = df1[i].split()
    b = listToString(a[1:])
    df_list1.append(b)
  if len(df1[i].split())==2:
    a = df1[i].split()
    b = listToString(a[1:])
    df_list1.append(b)


In [ ]:
mpg['manufacturer']=df_list
mpg['model']=df_list1

In [ ]:
ans=mpg.drop(columns = ['name'])

In [ ]:
ans

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,manufacturer,model
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet,chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick,skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth,satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc,rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford,torino
...,...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790,15.6,82,usa,ford,mustang gl
394,44.0,4,97.0,52.0,2130,24.6,82,europe,vw,pickup
395,32.0,4,135.0,84.0,2295,11.6,82,usa,dodge,rampage
396,28.0,4,120.0,79.0,2625,18.6,82,usa,ford,ranger


In [ ]:
import pandas as pd

#3-C

#3-D

#3-E

In [ ]:
fin = ans[['mpg','manufacturer']]
fin['manufacturer'][0]

'chevrolet'

In [ ]:
a = mpg_model[mpg_model.values >=5].index
a

Index(['amc', 'audi', 'buick', 'chevrolet', 'chrysler', 'datsun', 'fiat',
       'ford', 'honda', 'mazda', 'mercury', 'plymouth', 'pontiac', 'renault',
       'toyota', 'volkswagen'],
      dtype='object', name='manufacturer')

In [ ]:
def top3_petal_length(df):
    return df.sort_values(by="petal_length", ascending=False)[:3]

In [ ]:
new_ = []
no = []
for i in range(len(fin)):
  if (fin['manufacturer'][i] in a) & (fin['manufacturer'][i] not in no):
    new_.append(fin['mpg'][i])
    no.append(fin['manufacturer'][i])

In [ ]:
answer = pd.DataFrame(new_,a)

In [ ]:
a = answer[0]
a.sort_values(ascending =False).head(5)

manufacturer
plymouth    30.0
ford        27.0
honda       26.0
toyota      26.0
fiat        24.0
Name: 0, dtype: float64